This file includes a simple use case of the module.

A timed automata system is read from an xml file, an edge constraint is changed and the system is written back.

In [ ]:
from pyutap import *

system = UTAP.TimedAutomataSystem() # Create a system
parseXMLFile("examples/literature/CAS.xml", system, True) # Read system from file, 0 indicates success.

In [ ]:
templates = system.getTemplates() # Getting templates of the system

states_with_invariants = []

for template in templates:
        for state in template.states:
                if state.invariant.getSize() > 0: # Access states which have an invariant
                        states_with_invariants.append(state)
                        print("State uid:", state.uid) # Get state name
                        print("Invariant:", state.invariant) # Get state invariant
                        # print(state.toString()) # Print state in a readable form

In [ ]:
# Getting the edge guards of the system
Constants = UTAP.Constants

edge_guards_with_clock = []
for template in templates:
    for edge in template.edges:
        if (edge.guard.usesClock()): # Add guards that uses clocks to a list
            print("Source:", edge.src.uid, "; Dest:", edge.dst.uid)
            if edge.sync.getSize() > 0:
                print("Edge sync:", edge.sync)

            if edge.assign.getSize() > 0:
                print("Edge assign:", edge.assign)
                
            edge_guards_with_clock.append(edge.guard)

print(edge_guards_with_clock[0])
if edge_guards_with_clock[0].getKind() == Constants.AND:
    print("Guard has kind AND")

# Accessing subexpressions of expressions
print("Guard has", edge_guards_with_clock[0].getSize(), "subexpressions")
for i in range(edge_guards_with_clock[0].getSize()):
    print("Subexpression", i, edge_guards_with_clock[0][i])

In [ ]:
# Changing expression values
position = edge_guards_with_clock[0][0][1].getPosition() # Position of the expression
t = edge_guards_with_clock[0][0][1] # Sub-subexpression of the guard expression
if t.getKind() == Constants.CONSTANT:
    print("Const. old t=" + str(t.getValue()))
    #print("Double old t=" + str(t.getDoubleValue()))

exp = t.createConstant(100, position)
#exp = t.createDouble(100, position) # Creating double type expression
edge_guards_with_clock[0][0][1] = exp
if t.getKind() == Constants.CONSTANT:
    print("Const. new t=" + str(t.getValue()))
    #print("Double new t=" + str(t.getDoubleValue()))

In [ ]:
writeXMLFile("example.xml", system) # Write the system back to a file, 0 indicates success.

Cells above demonstrated the basic use of the wrapper, using the functions and classes generated directly from UTAP library. For further usage please refer to its documentation. From now on, usage of some helper classes and functions that this library has for itself will be shown.

In [1]:
# Changing expression values with helper classess and functions
from pyutap import *
from pyutap.nta import *

nta = NTAHelper("examples/literature/CAS.xml", "nta") # Get the system with helper class

edge_guards_with_clock = []
for template in nta.templates:
	for edge in template.edges:
		if edge.guard.usesClock():
			edge_guards_with_clock.append(edge.guard)

print(edge_guards_with_clock[0][0].toString())

e <= 300


In [2]:
change_expression_value(edge_guards_with_clock[0][0], 100)
print(edge_guards_with_clock[0][0].toString())

e <= 100


In [3]:
from pyutap.path_analysis import *

path = []
for template in nta.templates:
        print(template.edges[0].src.uid, template.edges[0].dst.uid)
        path.append(template.edges[0])

print(find_used_clocks(path))
print(path_exists(path))

(location) id2 (location) id1
(location) id3 (location) id7
(location) l7 (location) l6
(location) id9 (location) id8
(location) id11 (location) id10
{b'f'}
False


In [4]:
nta.writeToXML()